In [1]:
%pip uninstall -y lsdb hipscat

# %pip install git+http://hipscat.rubin.science
# %pip install git+http://lsdb.rubin.science

%pip install 'lsdb<0.4'

Note: you may need to restart the kernel to use updated packages.
  Using cached lsdb-0.3.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached hipscat-0.3.9-py3-none-any.whl.metadata (6.0 kB)
  Using cached nested_dask-0.2.1-py3-none-any.whl.metadata (5.0 kB)
  Using cached nested_pandas-0.2.2-py3-none-any.whl.metadata (4.1 kB)
  Using cached aiohappyeyeballs-2.4.3-py3-none-any.whl.metadata (6.1 kB)


Using cached lsdb-0.3.0-py3-none-any.whl (68 kB)
Using cached hipscat-0.3.9-py3-none-any.whl (79 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 MB 55.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.6/727.6 kB 16.9 MB/s eta 0:00:00
Using cached nested_dask-0.2.1-py3-none-any.whl (21 kB)
Using cached nested_pandas-0.2.2-py3-none-any.whl (33 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 56.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 53.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 56.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13

In [2]:
WITH_GAIA = True

In [3]:
catalog_name = 'ztf_ps1'
if WITH_GAIA:
    catalog_name += '_gaia'

In [4]:
%%time

from pathlib import Path

import dask.distributed
import lsdb
import pyarrow as pa
import pyarrow.dataset
import pyarrow.parquet

CPU times: user 27.3 s, sys: 2.17 s, total: 29.5 s
Wall time: 22.4 s


In [5]:
%%time

# HIPSCAT_PATH = Path('/ocean/projects/phy210048p/shared/hipscat/catalogs/')
HIPSCAT_PATH = Path('/data3/epyc/data3/hipscat/catalogs/')

PS1_BANDS = 'riz'

ztf_dr17_coord = lsdb.read_hipscat(
    'hipscat/ztf_dr17_coord',
    margin_cache='hipscat/ztf_dr17_coord_2arcsec',
).query('filter == 2')  # r-band only
gaia_distances = lsdb.read_hipscat(
    HIPSCAT_PATH / 'gaia_dr3' / 'gaia_edr3_distances',
    margin_cache = str(HIPSCAT_PATH / 'gaia_dr3' / 'gaia_edr3_distances_10arcs'),
    # Remove RA and Dec - we have them from the main Gaia catalog
    columns=[
        'source_id',
        'r_med_geo', 'r_lo_geo', 'r_hi_geo',
        'r_med_photogeo', 'r_lo_photogeo', 'r_hi_photogeo',
        'flag',
    ]
)
gaia = lsdb.read_hipscat(
    HIPSCAT_PATH / 'gaia_dr3' / 'gaia',
    # We must do str() due to
    # https://github.com/astronomy-commons/lsdb/issues/380
    margin_cache=str(HIPSCAT_PATH / 'gaia_dr3' / 'gaia_10arcs'),
    columns=[
        'ra', 'dec',
        'source_id',
        'ruwe',
        'parallax', 'parallax_over_error',
        'pmra', 'pmdec', 'pmra_error', 'pmdec_error',
        'teff_gspphot', 'teff_gspphot_lower', 'teff_gspphot_upper',
        'logg_gspphot', 'logg_gspphot_lower', 'logg_gspphot_upper',
        # 'ag_gspphot', 'ag_gspphot_lower', 'ag_gspphot_upper',
    ],
).query(
    "teff_gspphot_upper < 3800"
    " and parallax_over_error > 3"

    # " and ruwe < 1.4"
    # "parallax_over_error > 10.0"
    # " and teff_gspphot_upper < 3800"
    # " and (teff_gspphot_upper - teff_gspphot_lower) < 400"
    # " and logg_gspphot_lower > 4.5"
    # " and (logg_gspphot_upper - logg_gspphot_lower) < 0.2"
)
panstarrs = lsdb.read_hipscat(
    HIPSCAT_PATH / 'ps1' / 'ps1_otmo',
    margin_cache=str(HIPSCAT_PATH / 'ps1' / 'ps1_otmo_10arcs'),
    columns=
        ['raMean', 'decMean']
        + [f'{b}MeanPSFMag' for b in PS1_BANDS]
        + [f'{b}MeanPSFMagErr' for b in PS1_BANDS],
).query(
    "((rMeanPSFMag - iMeanPSFMag) + (rMeanPSFMagErr + iMeanPSFMagErr)) > 0.42"
    " and ((iMeanPSFMag - zMeanPSFMag) + (iMeanPSFMagErr + zMeanPSFMagErr)) > 0.23"
    " and rMeanPSFMagErr < 0.1 and iMeanPSFMagErr < 0.1 and zMeanPSFMagErr < 0.1"
)

/astro/users/kmalanch/.virtualenvs/default/lib/python3.10/site-packages/hipscat/io/file_io/file_io.py:320: UserWarning: point_map.fits file written in RING ordering, due to https://github.com/astronomy-commons/hipscat/issues/271. Converting to NESTED.
  warnings.warn(


CPU times: user 10.3 s, sys: 858 ms, total: 11.2 s
Wall time: 11.9 s


In [6]:
%%time

with dask.distributed.Client(
        n_workers=64,
        memory_limit='4GB',
        threads_per_worker=1
) as client:
    display(client)
    print(ztf_dr17_coord['oid'].shape[0].compute())

/astro/users/kmalanch/.virtualenvs/default/lib/python3.10/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38491 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:38491/status,
Dashboard: http://127.0.0.1:38491/status,Workers: 64
Total threads: 64,Total memory: 238.42 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38843,Workers: 64
Dashboard: http://127.0.0.1:38491/status,Total threads: 64
Started: Just now,Total memory: 238.42 GiB
Comm: tcp://127.0.0.1:46388,Total threads: 1
Dashboard: http://127.0.0.1:32930/status,Memory: 3.73 GiB
Nanny: tcp://127.0.0.1:39548,


2274767552


2024-11-01 11:06:44,460 - distributed.nanny - WARNING - Worker process still alive after 4.0 seconds, killing
2024-11-01 11:06:44,473 - distributed.nanny - WARNING - Worker process still alive after 4.0 seconds, killing
2024-11-01 11:06:44,475 - distributed.nanny - WARNING - Worker process still alive after 4.0 seconds, killing
2024-11-01 11:06:44,478 - distributed.nanny - WARNING - Worker process still alive after 4.0 seconds, killing
2024-11-01 11:06:44,481 - distributed.nanny - WARNING - Worker process still alive after 4.0 seconds, killing
2024-11-01 11:06:44,484 - distributed.nanny - WARNING - Worker process still alive after 4.0 seconds, killing
2024-11-01 11:06:44,486 - distributed.nanny - WARNING - Worker process still alive after 4.0 seconds, killing
2024-11-01 11:06:44,489 - distributed.nanny - WARNING - Worker process still alive after 4.0 seconds, killing
2024-11-01 11:06:44,491 - distributed.nanny - WARNING - Worker process still alive after 4.0 seconds, killing
2024-11-01

CPU times: user 7min 10s, sys: 4min 54s, total: 12min 4s
Wall time: 14min 49s


In [7]:
%%time

if WITH_GAIA:
    catalog = ztf_dr17_coord.crossmatch(
        gaia,
        radius_arcsec=1,
        n_neighbors=1,
        suffixes=['', '_gaia'],
    )
else:
    catalog = ztf_dr17_coord
    
catalog = catalog.crossmatch(
    panstarrs,
    radius_arcsec=1,
    n_neighbors=1,
    suffixes=['', ''],
)

# We must do join after cross-match to not lose margin cache due to
# https://github.com/astronomy-commons/lsdb/issues/382
if WITH_GAIA:
    catalog = catalog.join(gaia_distances, left_on='source_id_gaia', right_on='source_id', suffixes=['', '_edr3dist'])

CPU times: user 35min 37s, sys: 3min 36s, total: 39min 13s
Wall time: 39min 14s


In [ ]:
%%time

with dask.distributed.Client(
        n_workers=16,
        memory_limit='16GB',
        threads_per_worker=1
) as client:
    display(client)
    catalog.to_hipscat(catalog_name)

/astro/users/kmalanch/.virtualenvs/default/lib/python3.10/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 45429 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:45429/status,
Dashboard: http://127.0.0.1:45429/status,Workers: 16
Total threads: 16,Total memory: 238.42 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:37522,Workers: 16
Dashboard: http://127.0.0.1:45429/status,Total threads: 16
Started: Just now,Total memory: 238.42 GiB
Comm: tcp://127.0.0.1:45648,Total threads: 1
Dashboard: http://127.0.0.1:35622/status,Memory: 14.90 GiB
Nanny: tcp://127.0.0.1:45399,


/astro/users/kmalanch/.virtualenvs/default/lib/python3.10/site-packages/distributed/client.py:3361: UserWarning: Sending large graph of size 44.29 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


In [11]:
%%time

dataset = pyarrow.dataset.parquet_dataset(
    Path(catalog_name) / '_metadata',
).sort_by('oid')
pa.dataset.write_dataset(
    dataset,
    f"{catalog_name}_sorted",
    format='parquet',
    partitioning=pyarrow.dataset.partitioning(
        flavor="filename",
        schema=pyarrow.schema([dataset.schema.field('fieldid')]),
    ),
)

CPU times: user 8min 12s, sys: 3min 15s, total: 11min 27s
Wall time: 3min 45s
